In [1]:
# ONLY import lightweight packages
import os, sys
import subprocess
import importlib
import pandas as pd
import numpy as np
import json
import gc
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print("Base packages loaded.")

Base packages loaded.


In [2]:
try:
    subprocess.run(["ffmpeg", "-version"], check=True)
    print("FFmpeg is available to Python!")
    import whisper
except FileNotFoundError:
    print("FFmpeg is NOT visible to Python.")

ffmpeg version N-119416-g1dbc5675c1-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2025 the FFmpeg developers
built with Apple clang version 17.0.0 (clang-1700.0.13.3)
configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libharfbuzz --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxml2 --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-ver

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import tokenizer_from_json
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


In [4]:
import Whisper_functions as wp
import Model_Deployment_functions as mdf
import Result_functions as rf
import compare_models as cm

/Users/tim/APA_CLEAN/WD_v5
ffmpeg version N-119416-g1dbc5675c1-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2025 the FFmpeg developers
built with Apple clang version 17.0.0 (clang-1700.0.13.3)
configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libharfbuzz --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxml2 --enable-libxvid --enable-libzimg --enable-libzmq --

/opt/anaconda3/envs/APA_CLEAN/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Import summary file
audio_summary = wp.get_audio_metrics() # Audio summary is the benchmark file

# Stage 1: Import text files and transcribe
CER_errorrate, transcribed_texts, true_texts = wp.benchmark_speech_to_text(audio_summary) # gives error rate and texts in list form

Processing files: 100%|██████████| 20/20 [00:50<00:00,  2.52s/it]


In [13]:
# MAIN PIPELINE
importlib.reload(rf)
importlib.reload(mdf)

# Define models to be runned

model_type_1 = 'Emotion'
model_type_2 = 'Sentiment'
model_name_list_emotion = ['Binary2gram'] 
model_name_list_sentiment = ['Binary2gram']


########### MODEL DEPLOYMENT ###########

# Run emotion models
instances_emotion = mdf.create_model_instances(model_type = model_type_1, model_name_list = model_name_list_emotion, debugg_mode ='N') # gives back a dict of instances (model class), Debugg mode prints out more steps
mdf.full_deployment(instances_emotion, transcribed_texts, audio_summary) # Full deployment of the model, returns nothing. CSVs for each model in the model lists are stored in the model results folder

# Run sentiment models
instances_sentiment = mdf.create_model_instances(model_type = model_type_2, model_name_list = model_name_list_sentiment, debugg_mode ='N') # gives back a dict of instances (model class), Debugg mode prints out more steps
mdf.full_deployment(instances_sentiment, transcribed_texts, audio_summary) # Full deployment of the model, returns nothing. CSVs for each model in the model lists are stored in the model results folder


Stage 1 done! Created 1 directories! Model instances can be created.
Created directories : [['Binary2gram_Emotion', '/Users/tim/APA_CLEAN/WD_v5/Emotion_Model/Binary2gram/Binary2gram.h5', '/Users/tim/APA_CLEAN/WD_v5/Emotion_Model/Binary2gram/Binary2gram_config.json', '/Users/tim/APA_CLEAN/WD_v5/Emotion_Model/Binary2gram/Binary2gram_vocab.json']]
          
Stage 2 done! Created 1 model application instances! Tokenzing and predicting can start!

Running classification tasks
Running model: Binary2gram_Emotion
Saved result csv for Binary2gram_Emotion
Model run completed.
Stage 1 done! Created 1 directories! Model instances can be created.
Created directories : [['Binary2gram_Sentiment', '/Users/tim/APA_CLEAN/WD_v5/Sentiment_Model/Binary2gram/Binary2gram.h5', '/Users/tim/APA_CLEAN/WD_v5/Sentiment_Model/Binary2gram/Binary2gram_config.json', '/Users/tim/APA_CLEAN/WD_v5/Sentiment_Model/Binary2gram/Binary2gram_vocab.json']]
          
Stage 2 done! Created 1 model application instances! Tokenzi

In [14]:
# CLAUDE INTEGRATION AND COMPARISON
importlib.reload(cm)  # Reload if you make changes

# Run the comparison with the API key already in the code
comparison_df = cm.create_single_comparison_file()

# Display the first few rows of the comparison
comparison_df.head()

Using benchmark file: /Users/tim/APA_CLEAN/WD_v5/Whisper_Model/Audios/audio_memos.csv
Found 2 Emotion model files:
 - Binary2gram_Emotion: /Users/tim/APA_CLEAN/WD_v5/Model_results/Binary2gram_Emotion.csv
 - Binary2gram_Emotion: /Users/tim/APA_CLEAN/WD_v5/Model_results/Binary2gram_Emotion.csv
Processing emotion model: Binary2gram_Emotion from /Users/tim/APA_CLEAN/WD_v5/Model_results/Binary2gram_Emotion.csv
Successfully processed emotion model: Binary2gram_Emotion
Processing emotion model: Binary2gram_Emotion from /Users/tim/APA_CLEAN/WD_v5/Model_results/Binary2gram_Emotion.csv
Successfully processed emotion model: Binary2gram_Emotion
Found 2 Sentiment model files:
 - Binary2gram_Sentiment: /Users/tim/APA_CLEAN/WD_v5/Model_results/Binary2gram_Sentiment.csv
 - Binary2gram_Sentiment: /Users/tim/APA_CLEAN/WD_v5/Model_results/Binary2gram_Sentiment.csv
Processing sentiment model: Binary2gram_Sentiment from /Users/tim/APA_CLEAN/WD_v5/Model_results/Binary2gram_Sentiment.csv
Successfully process

,text,ground_truth_emotion,ground_truth_sentiment,Binary2gram_Emotion_prediction,Binary2gram_Emotion_accuracy,Binary2gram_Sentiment_prediction,Binary2gram_Sentiment_accuracy,Claude_Emotion_prediction,Claude_Sentiment_prediction,Claude_Emotion_accuracy,Claude_Sentiment_accuracy,Binary2gram_Emotion_claude_similarity,Binary2gram_Sentiment_claude_similarity
0,"Wow, I watched your demo and I’m truly impress...",admiration,positive,admiration,1.0,negative,0.0,admiration,positive,1.0,1.0,1.0,0.0
1,You wouldn’t believe it—my cat tried to fit in...,amusement,positive,neutral,0.0,negative,0.0,amusement,positive,1.0,1.0,0.0,0.0
2,I’m really upset that the report wasn’t finish...,anger,negative,disappointment,0.0,negative,1.0,disappointment,negative,0.0,1.0,1.0,1.0
3,"Honestly, it’s irritating when I have to repea...",annoyance,negative,neutral,0.0,negative,1.0,annoyance,negative,1.0,1.0,0.0,1.0
4,That’s exactly the approach we needed; I fully...,approval,positive,approval,1.0,negative,0.0,approval,positive,1.0,1.0,1.0,0.0
